In [42]:
def quick_sort(list):
    list5 = []
    if list == []:
        return []
    else:
        first = list[0]
        left = quick_sort([l for l in list[1:]if l < first])
        right = quick_sort([l for l in list[1:] if l > first])
        list1 = left +[first] + right
        for i in list1:
            list5.append(str(i))
        return list5

In [43]:
from clipper_admin import ClipperConnection, DockerContainerManager                                                                                                                                     
from clipper_admin.deployers import python as python_deployer     

In [44]:
clipper_conn = ClipperConnection(DockerContainerManager()) 

In [45]:
clipper_conn.start_clipper()
# Note that if the cluster is already created by start_clipper(), you can just connect to it using clipper_conn.connect() function.

20-03-03:23:14:40 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-03-03:23:14:44 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmplnynwfa_.yml
20-03-03:23:14:45 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [46]:
python_deployer.create_endpoint(clipper_conn, "sort-quick", "integers", quick_sort,'-1',1,30000000) 

20-03-03:23:14:50 INFO     [clipper_admin.py:236] [default-cluster] Application sort-quick was successfully registered
20-03-03:23:14:50 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpjh2nhjktclipper
20-03-03:23:14:50 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-03-03:23:14:50 INFO     [python.py:192] Python closure saved
20-03-03:23:14:50 INFO     [python.py:206] Using Python 3.6 base image
20-03-03:23:14:50 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpjh2nhjktclipper
20-03-03:23:14:51 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:0.4.1
20-03-03:23:14:51 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
20-03-03:23:14:51 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmpjh2nhjktclipper /model/
20-03-03:23:14:51 INFO     [clipper_admin.py:539] [default-cluster]  ---> eeae78c5fa99
20-03-

In [47]:
def predict(addr, x, batch=False):
    url = "http://%s/sort-quick/predict" % addr

    if batch:
        req_json = json.dumps({'input_batch': x})
    else:
        req_json = json.dumps({'input': list(x)})

    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))

In [48]:
import json ,requests
from datetime import datetime
addr = "127.0.0.1:1337"
# x = '{"input": [1.1, 2.2, 3.3]}'
x = [9,2,6,4]
predict(addr,x,batch=False)

'{"query_id":0,"output":-1,"default":true,"default_explanation":"Failed to retrieve a prediction response within the specified latency SLO"}', 30006.488000 ms


In [40]:
clipper_conn.stop_all()

20-03-03:21:39:42 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers


这是 用 clipper实现 快排  现在有点怀疑 clipper是不是 不能对列表排序